In [ ]:
# concatnation the data frames for the most active neurons recorded in each layer
import pandas as pd
import os
# First let read all metadata data frames and concatenate them into one with one extera column for the image root path and one for the image folder
data_root_batch1 = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062023\meta_data_files"
# read all metadata files in the folder and concatenate them into one
# the metadata files are saved in h5 format with the key "expriment_meta_data_df"
meta_data_df_batch1 = pd.DataFrame()
for file in os.listdir(data_root_batch1):
    if file.endswith(".h5") and "most" in file:
        meta_data_df_batch1 = pd.concat([meta_data_df_batch1, pd.read_hdf(os.path.join(data_root_batch1, file), key="expriment_meta_data_df")], axis=0)

# read other batch of metadata files
data_root_batch2 = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062123\meta_data_files"
# read all metadata files in the formt of "*_most_*.h5" folder and concatenate them into one
# the metadata files are saved in h5 format with the key "expriment_meta_data_df"
meta_data_df_batch2 = pd.DataFrame()
for file in os.listdir(data_root_batch2):
    # we want to read all data with that have most in their name
    if file.endswith(".h5") and "most" in file:
        metadata_df = pd.read_hdf((data_root_batch2+ "\\"+ file), key="expriment_meta_data_df")
        # each file is saved with the name of the data folder with a suffix of a random number at the end of the name taht
        # splited with "_" so we want to extract the data folder name from the file name and add it as a column to the metadata data frame
        folder_name = file.split("_")[1]
        for si in file.split("_")[2:-1]:
            folder_name = folder_name +("_"+si)

        metadata_df["data_root"] = os.path.join(r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062123",
                                                folder_name)
        # print humner of repited index in the metadata data frame
        print("number of repited index in the metadata data frame: ", metadata_df.index.duplicated().sum())
        meta_data_df_batch2 = pd.concat([meta_data_df_batch2, metadata_df], axis=0)

# concatenate the two batches of metadata data frames with adding a column for the image root path and one for the image folder 
# which is diffrent for each batch 
# first concatenate the two batches with tracking the batch number
meta_data_df_batch1["data_root"] = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062023"
# concatenate the two batches
metadata_df = pd.concat([meta_data_df_batch1, meta_data_df_batch2], axis=0)
# save the metadata data frame
saving_root = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"
metadata_df.to_hdf(os.path.join(saving_root, "metadata_most_df.h5"), key="metadata_df")

In [ ]:
# find the l2 distance the data frames for the most active neurons recorded in each layer
	 
import glob
import shutil
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
import torch
import numpy as np
import sys
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
from core.utils.CNN_scorers import TorchScorer
from inSilico_experiments.utils.pothook_analysis_lib import *



network_list = ['alexnet', 'vgg16', 'resnet50', 'resnet101']
# make a dict of modek name and models scorer
scorer_dict = dict()
for net_name in network_list:
    scorer_dict[net_name] = TorchScorer(net_name)
    metadata_df[net_name] = np.nan

# add a new column to the dataframe to store the l2 distance between the generated image and the target image
metadata_df["l2_distance"] = np.nan
row_count = 0
# loop over all rows of the meta data dataframe
for i, row in metadata_df.iterrows():
    if row["output_type"] == "best_gen_imgs_RF_masked":
        # load the image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i}.jpg"))[0]
        img = Image.open(img_path)
        gen_image_tensor = ToTensor()(img)
        # find target image id and load the image as tensor
        i_target = find_target_image_id(metadata_df, row)
        # load the target image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i_target}.jpg"))[0]
        img = Image.open(img_path)
        target_image_tensor = ToTensor()(img)    
        # plot the generated image and the target image in a figure to make sure that they are the same
        '''fig, ax = plt.subplots(1, 2)
        ax[0].imshow(ToPILImage()(gen_image_tensor))
        ax[1].imshow(ToPILImage()(target_image_tensor))
        plt.show()'''
        # calculate the l2 distance between the generated image and the target image and store it in the dataframe
        metadata_df.loc[i, "l2_distance"] = l2_distance(gen_image_tensor, target_image_tensor).item()
        #get_similarty_score_by_CNN(gen_image_tensor, target_image_tensor, scorer_dict, metadata_df, i)
        row_count = row_count + 1
        
metadata_df.to_hdf(os.path.join(saving_root, "metadata_most_df_with_similarity.h5"), key="metadata_df")

In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
from inSilico_experiments.utils.CosineDataset import *
from inSilico_experiments.utils.pothook_analysis_lib import *


# implimenting torch data loader
data_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"
metadata_df_most = pd.read_hdf(os.path.join(data_path, "metadata_most_df.h5"), key="metadata_df")



layer_short_list= metadata_df_most["layer_short"].unique()
# remove the NaN values from the layer_short_list
layer_short_list = layer_short_list[~pd.isnull(layer_short_list)]
similarity_metric_list = metadata_df_most["similarity_metric"].unique()
# remove the NaN values from the similarity_metric_list
similarity_metric_list = similarity_metric_list[~pd.isnull(similarity_metric_list)]

pop_size_list = metadata_df_most["pop_size"].unique()
# remove the NaN values from the pop_size_list
pop_size_list = np.sort(pop_size_list[~pd.isnull(pop_size_list)])

gen_rerun_id_list = metadata_df_most["gen_rerun_id"].unique()
# remove the NaN values from the gen_rerun_id_list
gen_rerun_id_list = np.sort(gen_rerun_id_list[~pd.isnull(gen_rerun_id_list)])

pop_resampling_id_list = metadata_df_most["pop_resampling_id"].unique()
# remove the NaN values from the pop_resampling_id_list
pop_resampling_id_list = np.sort(pop_resampling_id_list[~pd.isnull(pop_resampling_id_list)])

smpling_type_list = metadata_df_most["sub_pop_type"].unique()
# remove the NaN values from the smpling_type_list
smpling_type_list = smpling_type_list[~pd.isnull(smpling_type_list)]

# loop over pop_size_list and sampling_type and layer_short_list and similarity_metric_list 

for layer_short in layer_short_list:
    for similarity_metric in similarity_metric_list:
        for pop_size in pop_size_list:
            for sampling_type in smpling_type_list:
                sub_meta_df = metadata_df_most[\
                    (metadata_df_most["output_type"] == "best_gen_imgs_RF_masked") & (
                    metadata_df_most["layer_short"]==layer_short) & (
                    metadata_df_most["sub_pop_type"] == smpling_type) & (
                    metadata_df_most["similarity_metric"] == similarity_metric) & ( 
                    metadata_df_most["trget_imge_name"] == image_name)]


In [ ]:

layer_short_list= metadata_df_most["layer_short"].unique()
# remove the NaN values from the layer_short_list
similarity_metric = "cosine"
pop_size = 32
gen_rerun_id = 0
output_type = "best_gen_imgs_RF_masked"
RF_treshold = 2
pop_resampling_id = 0
smpling_type = "random"
sub_meta_df = metadata_df_most[\
                (metadata_df_most["output_type"] == "best_gen_imgs_RF_masked") & (
                metadata_df_most["layer_short"]==layer_short) & (
                metadata_df_most["sub_pop_type"] == smpling_type) & (
                metadata_df_most["similarity_metric"] == similarity_metric) & ( 
                metadata_df_most["trget_imge_name"] == image_name)]
path_list = [os.path.join(sub_meta_df["data_root"][i], f"{sub_meta_df.index[i]}.jpg") for i in range(len(sub_meta_df))]
label_list = sub_meta_df.index.values

transform = Compose([
    ToTensor()
])

dataset = SimpleDataset(path_list, label_list, transform=transform)
dataloader = DataLoader(dataset, batch_size=10, num_workers=5)

for i, data in enumerate(dataloader):
    print(i, data[1])

In [ ]:
# concatnation the data frames for the random active neurons recorded in each layer

import pandas as pd
import os
# First let read all metadata data frames and concatenate them into one with one extera column for the image root path and one for the image folder
# read other batch of metadata files
data_root_batch = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062123\meta_data_files"
# read all metadata files in the formt of "*_most_*.h5" folder and concatenate them into one
# the metadata files are saved in h5 format with the key "expriment_meta_data_df"
meta_data_df_batch = pd.DataFrame()
for file in os.listdir(data_root_batch):
    # we want to read all data with that have most in their name
    if file.endswith(".h5") and "random" in file:
        #print(f"reading file {file}")
        metadata_df = pd.read_hdf((data_root_batch+ "\\"+ file), key="expriment_meta_data_df")
        # each file is saved with the name of the data folder with a suffix of a random number at the end of the name taht
        # splited with "_" so we want to extract the data folder name from the file name and add it as a column to the metadata data frame
        folder_name = file.split("_")[1]
        for si in file.split("_")[2:-1]:
            folder_name = folder_name +("_"+si)

        metadata_df["data_root"] = os.path.join(r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\O2_cluster_exp_062123",
                                                folder_name)
        print("number of repited index in the metadata data frame: ", metadata_df.index.duplicated().sum())
        meta_data_df_batch = pd.concat([meta_data_df_batch, metadata_df], axis=0)

metadata_df = meta_data_df_batch
# save the metadata data frame

saving_root = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"
metadata_df.to_hdf(os.path.join(saving_root, "metadata_random_df.h5"), key="metadata_df")

In [ ]:
# find the l2 distance the data frames for the random  active neurons recorded in each layer 
import glob
import shutil
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
import torch
import numpy as np
import sys
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
from core.utils.CNN_scorers import TorchScorer
from inSilico_experiments.utils.pothook_analysis_lib import *



#network_list = ['alexnet', 'vgg16', 'resnet50', 'resnet101']
# make a dict of modek name and models scorer
#scorer_dict = dict()
#for net_name in network_list:
#    scorer_dict[net_name] = TorchScorer(net_name)
#    metadata_df[net_name] = np.nan

# add a new column to the dataframe to store the l2 distance between the generated image and the target image
metadata_df["l2_distance"] = np.nan
row_count = 0
# loop over all rows of the meta data dataframe
for i, row in metadata_df.iterrows():
    if row["output_type"] == "best_gen_imgs_RF_masked":
        # load the image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i}.jpg"))[0]
        img = Image.open(img_path)
        gen_image_tensor = ToTensor()(img)
        # find target image id and load the image as tensor
        i_target = find_target_image_id(metadata_df, row)
        # load the target image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i_target}.jpg"))[0]
        img = Image.open(img_path)
        target_image_tensor = ToTensor()(img)    
        # plot the generated image and the target image in a figure to make sure that they are the same
        '''fig, ax = plt.subplots(1, 2)
        ax[0].imshow(ToPILImage()(gen_image_tensor))
        ax[1].imshow(ToPILImage()(target_image_tensor))
        plt.show()'''
        # calculate the l2 distance between the generated image and the target image and store it in the dataframe
        try:
            metadata_df.loc[i, "l2_distance"] = l2_distance(gen_image_tensor, target_image_tensor).item()
        except:
            print(f"error in calculating l2 distance for raw: {i}")
        #get_similarty_score_by_CNN(gen_image_tensor, target_image_tensor, scorer_dict, metadata_df, i)
        row_count = row_count + 1
        
metadata_df.to_hdf(os.path.join(saving_root, "metadata_random_df_with_similarity.h5"), key="metadata_df")

In [ ]:
# concatnation the data frames for the neurons recorded across different layers
import pandas as pd
import os
# First let read all metadata data frames and concatenate them into one with one extera column for the image root path and one for the image folder
# read other batch of metadata files
data_root_batch = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\cross_layer_recording_o2_062823\meta_data_files"
# read all metadata files in the formt of "*_most_*.h5" folder and concatenate them into one
# the metadata files are saved in h5 format with the key "expriment_meta_data_df"
meta_data_df_batch = pd.DataFrame()
for file in os.listdir(data_root_batch):
    # we want to read all data with that have most in their name
    if file.endswith(".h5"):
        #print(f"reading file {file}")
        metadata_df = pd.read_hdf((data_root_batch+ "\\"+ file), key="expriment_meta_data_df")
        # each file is saved with the name of the data folder with a suffix of a random number at the end of the name taht
        # splited with "_" so we want to extract the data folder name from the file name and add it as a column to the metadata data frame
        folder_name = file.split("_")[1]
        for si in file.split("_")[2:-1]:
            folder_name = folder_name +("_"+si)

        metadata_df["data_root"] = os.path.join(r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\raw_data\cross_layer_recording_o2_062823",
                                                folder_name)
        print("number of repited index in the metadata data frame: ", metadata_df.index.duplicated().sum())
        meta_data_df_batch = pd.concat([meta_data_df_batch, metadata_df], axis=0)

metadata_df = meta_data_df_batch
# save the metadata data frame

saving_root = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"
metadata_df.to_hdf(os.path.join(saving_root, "metadata_cross_layer_recording_df.h5"), key="metadata_df")

In [ ]:
# 
import glob
import shutil
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
import torch
import numpy as np
import sys
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
from core.utils.CNN_scorers import TorchScorer
from inSilico_experiments.utils.pothook_analysis_lib import *



# add a new column to the dataframe to store the l2 distance between the generated image and the target image
metadata_df["l2_distance"] = np.nan
row_count = 0
# loop over all rows of the meta data dataframe
for i, row in metadata_df.iterrows():
    if row["output_type"] == "best_gen_imgs_RF_masked":
        # load the image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i}.jpg"))[0]
        img = Image.open(img_path)
        gen_image_tensor = ToTensor()(img)
        # find target image id and load the image as tensor
        i_target = find_target_image_id(metadata_df, row)
        # load the target image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i_target}.jpg"))[0]
        img = Image.open(img_path)
        target_image_tensor = ToTensor()(img)    
        # plot the generated image and the target image in a figure to make sure that they are the same
        '''fig, ax = plt.subplots(1, 2)
        ax[0].imshow(ToPILImage()(gen_image_tensor))
        ax[1].imshow(ToPILImage()(target_image_tensor))
        plt.show()'''
        # calculate the l2 distance between the generated image and the target image and store it in the dataframe
        try:
            metadata_df.loc[i, "l2_distance"] = l2_distance(gen_image_tensor, target_image_tensor).item()
        except:
            print(f"error in calculating l2 distance for raw: {i}")
        #get_similarty_score_by_CNN(gen_image_tensor, target_image_tensor, scorer_dict, metadata_df, i)
        row_count = row_count + 1
        
metadata_df.to_hdf(os.path.join(saving_root, "metadata_cross_layer_recording_with_similarity.h5"), key="metadata_df")

In [1]:
import glob
import shutil
from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage
import torch
import numpy as np
import sys
sys.path.append(r"C:\Users\Alireza\Documents\Git\Cosine-Project")
from inSilico_experiments.utils.func_lib import *
from core.utils.CNN_scorers import TorchScorer
from inSilico_experiments.utils.pothook_analysis_lib import *

data_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed"

metadata_df_cosine_intralayer = pd.read_hdf(os.path.join(data_path, "metadata_df_inter_layer_with_normalized_dist.h5"), key="metadata_df")
metadata_df_cosine_crosslayer = pd.read_hdf(os.path.join(data_path, "metadata_df_intera_layer_with_normalized_dist.h5"), key="metadata_df")
# concatenate the two dataframes
metadata_df = pd.concat([metadata_df_cosine_intralayer, metadata_df_cosine_crosslayer], ignore_index=False)
# concatenate the two dataframes

RF_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed\rf_filters"

init_img_path = r"N:\PonceLab\Users\Alireza\insilico_experiments\Alexnet_remonstration_across_different_layer_062023\post_processed\init_img\init_img.jpg"
init_img = Image.open(init_img_path)
init_img = ToTensor()(init_img)

# add a new column to the dataframe to store the l2 distance between the generated image and the target image
metadata_df["sim_index"] = np.nan
row_count = 0
# loop over all rows of the meta data dataframe
for i, row in metadata_df.iterrows():
    if row["output_type"] == "best_gen_imgs_RF_masked":
        # load the image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i}.jpg"))[0]
        img = Image.open(img_path)
        gen_image_tensor = ToTensor()(img)
        # find target image id and load the image as tensor
        i_target = find_target_image_id(metadata_df, row)
        # load the target image as tensor
        img_path = glob.glob(os.path.join(row["data_root"], f"{i_target}.jpg"))[0]
        img = Image.open(img_path)
        target_image_tensor = ToTensor()(img)    

        if row["layer_short"] == "conv5432":
            layer_short = "conv5"
            pop_size = row["pop_size"]*4
        elif row["layer_short"] == "conv53":
            layer_short = "conv5"
            pop_size = row["pop_size"]*2
        else:
            layer_short = row["layer_short"]
            pop_size = row["pop_size"]
        
        RF_map = np.load(os.path.join(RF_path, f"{layer_short}_{pop_size}.npz"))

        RF_filter = RF_map["fitmap"] > RF_map["fitmap"][int(RF_map["xo"]+(1.5*RF_map["sigma_x"])), int(RF_map["yo"]+(1.5*RF_map["sigma_y"]))]

        init_img_rf_masked =\
            (torch.from_numpy(np.absolute(RF_map["fitmap"][None,:,:])) / RF_map["fitmap"].max()) *\
                    init_img
        # calculate the l2 distance between the generated image and the target image and store it in the dataframe
        try:
            metadata_df.loc[i, "sim_index_l1"] = \
                        sim_index_l1(target_image_tensor, gen_image_tensor, init_img_rf_masked, RFfilter=np.array(RF_filter))
            metadata_df.loc[i, "sim_index_l2"] = \
                        sim_index_l2(target_image_tensor, gen_image_tensor, init_img_rf_masked, RFfilter=np.array(RF_filter))
        except:
            print(f"error in calculating l2 distance for raw: {i}")
        #get_similarty_score_by_CNN(gen_image_tensor, target_image_tensor, scorer_dict, metadata_df, i)
        row_count = row_count + 1
        if row_count % 1000 == 0:
            metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index_v2.h5"), key="metadata_df")
        
metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index_v2.h5"), key="metadata_df")
metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index_v2_final.h5"), key="metadata_df")

C:\Users\Alireza\AppData\Local\Temp\ipykernel_25564\2337153699.py:72: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['output_type', 'trget_imge_name', 'similarity_metric', 'gan_name',
       'layer_name', 'layer_short', 'net_name', 'img_size', 'pading_size',
       'input_size', 'pop_unit_idx', 'sub_pop_type', 'gen_rerun_id',
       'data_root'],
      dtype='object')]

  metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index_v2.h5"), key="metadata_df")
C:\Users\Alireza\AppData\Local\Temp\ipykernel_25564\2337153699.py:74: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['output_type', 'trget_imge_name', 'similarity_metric', 'gan_name',
       'layer_name', 'layer_short', 'net_name', 'img_siz

In [2]:
metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index.h5"), key="metadata_df")

C:\Users\Alireza\AppData\Local\Temp\ipykernel_12896\1268087068.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->Index(['output_type', 'trget_imge_name', 'similarity_metric', 'gan_name',
       'layer_name', 'layer_short', 'net_name', 'img_size', 'pading_size',
       'input_size', 'pop_unit_idx', 'sub_pop_type', 'gen_rerun_id',
       'data_root'],
      dtype='object')]

  metadata_df.to_hdf(os.path.join(data_path, "metadata_df_with_sim_index.h5"), key="metadata_df")


In [2]:
metadata_df.head()

,output_type,trget_imge_name,similarity_metric,pop_size,pop_resampling_id,gan_name,layer_name,layer_short,net_name,img_size,...,data_root,l2_distance,alexnet,vgg16,resnet50,resnet101,pixel_dist_normalized,sim_index,sim_index_l1,sim_index_l2
1687416128_4419577,target_img,imagenet_18,None,128,0,fc6,.features.Conv2d3,conv2,alexnet,None,...,N:\PonceLab\Users\Alireza\insilico_experiments...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1687416128_4445313,target_img_RF_masked,imagenet_18,None,128,0,fc6,.features.Conv2d3,conv2,alexnet,None,...,N:\PonceLab\Users\Alireza\insilico_experiments...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1687416147_5999399,best_gen_imgs,imagenet_18,cosine,128,0,fc6,.features.Conv2d3,conv2,alexnet,None,...,N:\PonceLab\Users\Alireza\insilico_experiments...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1687416147_8799687,best_gen_imgs_RF_masked,imagenet_18,cosine,128,0,fc6,.features.Conv2d3,conv2,alexnet,None,...,N:\PonceLab\Users\Alireza\insilico_experiments...,8.271968,NaN,NaN,NaN,NaN,0.540106,NaN,0.700716,0.656972
1687416147_6061125,last_gen_mean_imgs,imagenet_18,cosine,128,0,fc6,.features.Conv2d3,conv2,alexnet,None,...,N:\PonceLab\Users\Alireza\insilico_experiments...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
